In [196]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB

In [189]:
x_train, x_test, y_train, y_test = pd.read_csv('trainData.csv'), pd.read_csv('testData.csv'), pd.read_csv('trainLabels.csv'), pd.read_csv('testLabels.csv')


In [190]:
x_train = x_train.to_numpy().astype('float')
x_test = x_test.to_numpy().astype('float')

y_train = y_train.to_numpy().astype('float').squeeze()
y_test = y_test.to_numpy().astype('float').squeeze()


In [191]:
class_idx = [np.where(y_train == c)[0] for c in range(10)]
means = []
stds = []

for c in range(10):
  means.append(x_train[class_idx[c], :].mean(axis=0))
  stds.append(x_train[class_idx[c], :].std(axis=0))

means = np.array(means)
stds = np.array(stds)

u, c = np.unique(y_train, return_counts=True)
pc_s = c / c.sum()


In [192]:
norms = []
for cnt in range(x_test.shape[1]):
  cs = []
  for c in range(10):
    if stds[c, cnt] == 0:
      cs.append(np.ones(x_test.shape[0]))
    else:
      cs.append(norm.pdf(x_test[:, cnt], loc=means[c, cnt], scale=stds[c, cnt]))  
  
  norms.append(cs)
    



In [193]:
    
norms = np.array(norms).transpose((2, 1, 0))
print(norms.shape)
norms = norms.prod(axis = 2)

print(norms.shape)
norms = np.multiply(norms, pc_s)
print(norms.shape)
norms = norms.argmax(axis=1)


(2499, 10, 196)
(2499, 10)
(2499, 10)


In [199]:
acc = accuracy_score(y_test, norms)
f1 = f1_score(y_test, norms, average='macro')
print(acc, f1)

0.6410564225690276 0.6025507165301931


In [201]:
nb = GaussianNB()
preds = nb.fit(x_train, y_train).predict(x_test)
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds, average='macro')
print(acc, f1)


0.5070028011204482 0.47201625559743016
